In [2]:
import pandas as pd
import numpy as np

In [ ]:
# Leer datos
samples_df = pd.read_csv('./../data/samples.csv') 
busstop_df = pd.read_csv('./../data/allbusstop.csv')


In [4]:
print(f'Hay {busstop_df.shape[0]} paraderos.')
print(f'Hay {samples_df.shape[0]} puntos GPS.')

Hay 155 paraderos.
Hay 1685070 puntos GPS.


### 1. Cantidad de samples con 155 puntos

In [5]:
menos_de_155 = []
for id, df in samples_df.groupby(by='sample'):
    # 1° No evaluar los DF con pocos puntos - NOTA: tenemos 155 paraderos
    if (df.shape[0]>155):
        menos_de_155.append(id)
        
print(f'Hay {len(menos_de_155)} / {samples_df['sample'].unique().shape[0]} samples con menos de 155 puntos GPS')

Hay 3806 / 5015 samples con menos de 155 puntos GPS


### 2. 

# 2. Analisis de los registros de monitorizacion

Tareas a realizar:
- Filtrar los paraderos de vuelta (los que se usa y eliminar los que no se usan y que conincidan con el de ida)

Se da nuevo formato a la tabla, creando nuevas columnas y eliminando otras.

In [2]:
# cargar datos en bruto
monitoringdf = pd.read_csv('../data/_Process_monitoring__202303251137.csv', sep=',')
monitoringdf = monitoringdf[['id', 'latitude','longitude','date','time', 'lap', 'navigation', 'vehicle_id']]
monitoringdf

,id,latitude,longitude,date,time,lap,navigation,vehicle_id
0,13173398,-13.549597,-71.877510,2023-03-01,05:10:11,1,285,SJ39
1,13177467,-13.524529,-71.977527,2023-03-01,06:04:51,1,298,SJ34
2,13189674,-13.534183,-71.980709,2023-03-01,07:22:21,1,7,SJ29
3,13192729,-13.551146,-71.986928,2023-03-01,07:39:04,1,202,SJ21
4,13193272,-13.548642,-71.881010,2023-03-01,07:41:52,1,107,SJ34
...,...,...,...,...,...,...,...,...
1685065,17241845,-13.525440,-71.975079,2023-03-25,08:50:24,2,288,SJ24
1685066,17253753,-13.551121,-71.989621,2023-03-25,10:01:56,2,253,SJ16
1685067,17257644,-13.543207,-71.897785,2023-03-25,10:26:10,2,126,SJ24
1685068,17266468,-13.532774,-71.921865,2023-03-25,11:14:26,1,258,SJ43


# Cleaning

Mostramos la cantidad de datos, existen puntos que por error de lectura se leen dos veces, estos puntos se detectan si en las columnas datetime y latitud-longitud existen datos iguales, esto no debe suceder ya que el bus esta en constante movimiento como el tiempo. Como consecuencia se decide eliminar estos duplicados.

In [8]:
# DUPLICADOS
print(f'Shape dataset: {monitoringdf.shape}')

# Verificar y contar si hay puntos duplicados segun datetime y latitude_longitude
base = ['date', 'time', 'latitude', 'longitude']
twocols_dupli = monitoringdf.duplicated(subset=base)
print(f"Nro de duplicados en {base}: {twocols_dupli.sum()}")

# ============================ Solo para comprobacion ============================
# Verificar y contar si hay puntos duplicados segun latitude_longitude
# onecol_dupli =  monitoringdf['latitude_longitude'].duplicated()
# print(len(twocols_dupli), len(onecol_dupli))
# print(twocols_dupli.sum(), onecol_dupli.sum())
# print(abs(twocols_dupli.sum() - onecol_dupli.sum()))
# tmp = [ not x==y for x,y in zip(twocols_dupli, onecol_dupli)]
# data_tmp = monitoringdf.copy()
# data_tmp['dupli'] = tmp
# data_tmp[data_tmp['dupli'] == True]
# ================================================================================

# Eliminar columnas
monitoringdf.drop_duplicates(subset=base, inplace=True)
print(f"Duplicados eliminados => shape dataset: {monitoringdf.shape}")

Shape dataset: (1685070, 8)
Nro de duplicados en ['date', 'time', 'latitude', 'longitude']: 13116
Duplicados eliminados => shape dataset: (1671954, 8)


In [9]:
# NANs
# verificar si hay datos faltantes
print("Cantidad de NaNs:")
for col in monitoringdf:
    nans = monitoringdf[col].isna().sum()
    print(f"\tColumna {col}: {nans}")

Cantidad de NaNs:
	Columna id: 0
	Columna latitude: 0
	Columna longitude: 0
	Columna date: 0
	Columna time: 0
	Columna lap: 0
	Columna navigation: 0
	Columna vehicle_id: 0


In [10]:
# preparar el dataset para iterar en todos los posibles samples
samples_df = monitoringdf.copy()

# Crear columna sample para identificar los samples, utilizando la fecha vehiculo y lapso, todo ello encapsulado en un string
samples_df['sample'] = samples_df.apply(lambda row: f"{row['date']} {row['vehicle_id']} {row['lap']}" , axis=1)
samples_dict = {k:v for v, k in enumerate(samples_df['sample'].unique())}
samples_df['sample'] = samples_df['sample'].apply(lambda sample: samples_dict[sample])
samples_df.sort_values(by='sample', inplace=True)
print('nro posibles samples:', len(samples_dict))

# convertir date and time en el formato datetime
samples_df['datetime'] = samples_df['date'] + ' ' + samples_df['time']
samples_df['datetime'] = pd.to_datetime(samples_df['datetime'], format='%Y-%m-%d %H:%M:%S')
samples_df.drop(columns=['date', 'time'], inplace=True)

samples_df

nro posibles samples: 5015


,id,latitude,longitude,lap,navigation,vehicle_id,sample,datetime
104831,13181518,-13.522822,-71.969916,1,102,SJ39,0,2023-03-01 06:35:15
49342,13179415,-13.531396,-71.979872,1,12,SJ39,0,2023-03-01 06:20:32
27531,13178576,-13.545698,-71.983907,1,42,SJ39,0,2023-03-01 06:13:46
57321,13175650,-13.525765,-71.982937,1,111,SJ39,0,2023-03-01 05:44:47
25378,13174422,-13.530906,-71.929595,1,288,SJ39,0,2023-03-01 05:24:51
...,...,...,...,...,...,...,...,...
1679049,17019979,-13.553088,-71.863690,0,98,SJ32,5014,2023-03-24 05:13:08
1676807,17019886,-13.551734,-71.869753,0,101,SJ32,5014,2023-03-24 05:11:33
1677151,17019900,-13.551959,-71.868857,0,106,SJ32,5014,2023-03-24 05:11:45
1675441,17019830,-13.550220,-71.873981,0,194,SJ32,5014,2023-03-24 05:10:23


# Analisis de Samples

In [11]:
# Funcion para extraer un sample por id o nombre y csv para graficar
def getSample(id=-1, name='', to_csv=False):
    if(name != ''):
        id = samples_dict.get(name, -1)
    
    if (id != -1):
        tmp = samples_df[samples_df['sample'] == id].sort_values(by='datetime').reset_index(drop=True)
        print(tmp.shape)
        name = f"{str(tmp.loc[0,'datetime']).split()[0]} {tmp.loc[0,'vehicle_id']} {tmp.loc[0,'lap']}"
        print(tmp.to_string())
        if to_csv: 
            tmp.to_csv(f'{name}- {id} -sample.csv', index=False)
            
    else:
        print(f'No existe una muestra {id} o {name}')

Comprobamos si existen casillas vacias (Nans) en todo el dataframe.

In [12]:
# NANs
# verificar si hay datos faltantes
print("Cantidad de NaNs:")
for col in monitoringdf:
    nans = monitoringdf[col].isna().sum()
    print(f"\tColumna {col}: {nans}")

Cantidad de NaNs:
	Columna id: 0
	Columna latitude: 0
	Columna longitude: 0
	Columna date: 0
	Columna time: 0
	Columna lap: 0
	Columna navigation: 0
	Columna vehicle_id: 0


In [13]:
# preparar el dataset para iterar en todos los posibles samples
samples_df = monitoringdf.copy()

# Crear columna sample para identificar los samples, utilizando la fecha vehiculo y lapso, todo ello encapsulado en un string
samples_df['sample'] = samples_df.apply(lambda row: f"{row['date']} {row['vehicle_id']} {row['lap']}" , axis=1)
samples_dict = {k:v for v, k in enumerate(samples_df['sample'].unique())}
samples_df['sample'] = samples_df['sample'].apply(lambda sample: samples_dict[sample])
samples_df.sort_values(by='sample', inplace=True)
print('nro posibles samples:', len(samples_dict))

# convertir date and time en el formato datetime
samples_df['datetime'] = samples_df['date'] + ' ' + samples_df['time']
samples_df['datetime'] = pd.to_datetime(samples_df['datetime'], format='%Y-%m-%d %H:%M:%S')
samples_df.drop(columns=['date', 'time'], inplace=True)

samples_df

nro posibles samples: 5015


,id,latitude,longitude,lap,navigation,vehicle_id,sample,datetime
104831,13181518,-13.522822,-71.969916,1,102,SJ39,0,2023-03-01 06:35:15
49342,13179415,-13.531396,-71.979872,1,12,SJ39,0,2023-03-01 06:20:32
27531,13178576,-13.545698,-71.983907,1,42,SJ39,0,2023-03-01 06:13:46
57321,13175650,-13.525765,-71.982937,1,111,SJ39,0,2023-03-01 05:44:47
25378,13174422,-13.530906,-71.929595,1,288,SJ39,0,2023-03-01 05:24:51
...,...,...,...,...,...,...,...,...
1679049,17019979,-13.553088,-71.863690,0,98,SJ32,5014,2023-03-24 05:13:08
1676807,17019886,-13.551734,-71.869753,0,101,SJ32,5014,2023-03-24 05:11:33
1677151,17019900,-13.551959,-71.868857,0,106,SJ32,5014,2023-03-24 05:11:45
1675441,17019830,-13.550220,-71.873981,0,194,SJ32,5014,2023-03-24 05:10:23


In [14]:
# Funcion para extraer un sample por id o nombre y csv para graficar
def getSample(id=-1, name='', to_csv=False):
    if(name != ''):
        id = samples_dict.get(name, -1)
    
    if (id != -1):
        tmp = samples_df[samples_df['sample'] == id].sort_values(by='datetime').reset_index(drop=True)
        print(tmp.shape)
        name = f"{str(tmp.loc[0,'datetime']).split()[0]} {tmp.loc[0,'vehicle_id']} {tmp.loc[0,'lap']}"
        print(tmp.to_string())
        if to_csv: 
            tmp.to_csv(f'{name}- {id} -sample.csv', index=False)
    else:
        print(f'No existe una muestra {id} o {name}')

In [15]:
# cantidad de laps y vehiculos
veh = samples_df['vehicle_id'].unique()
lap = samples_df['lap'].unique()

print(len(veh))
print(sorted(veh))

print(len(lap))
print(sorted(lap))

43
['SJ01', 'SJ02', 'SJ03', 'SJ04', 'SJ05', 'SJ06', 'SJ07', 'SJ08', 'SJ09', 'SJ10', 'SJ11', 'SJ12', 'SJ13', 'SJ14', 'SJ15', 'SJ16', 'SJ17', 'SJ18', 'SJ19', 'SJ20', 'SJ21', 'SJ22', 'SJ23', 'SJ24', 'SJ25', 'SJ26', 'SJ27', 'SJ28', 'SJ29', 'SJ30', 'SJ31', 'SJ32', 'SJ33', 'SJ34', 'SJ35', 'SJ36', 'SJ37', 'SJ38', 'SJ39', 'SJ40', 'SJ41', 'SJ42', 'SJ43']
9
[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]


In [16]:
# cantidad de puntos por LAP
samples_df.groupby('lap').size()

lap
0     43941
1    352566
2    352893
3    342559
4    326027
5    207131
6     40302
7      5896
8       639
dtype: int64

In [17]:
# Nro de samples por LAP
cp = []
for lap in range(9):
    v = samples_df[samples_df['lap'] == lap]['sample'].unique().shape[0]
    cp.append(v)

# lap = indice de la lista
cp

[932, 911, 900, 847, 768, 528, 110, 17, 2]

In [18]:
# id de samples por LAP
samples_df[samples_df['lap'] == 7]['sample'].unique()

array([ 363,  904, 1031, 1258, 1795, 1820, 2150, 2817, 3073, 3234, 3305,
       3883, 3926, 3927, 4257, 4297, 4389])

In [21]:
samples_sizes = samples_df.groupby('sample').size().to_list()
print('Max:', max(samples_sizes))
print('Min:', min(samples_sizes))

# cantidad de samples por nro de puntos
count, bins = np.histogram(samples_sizes, bins=10, range=(0, 2400))
bins = np.round(bins, 0)
ranges = {f'{int(bins[i])}-{int(bins[i+1])-1}':int(count[i]) for i in range(len(count))}
ranges

Max: 2328
Min: 1


{'0-239': 1391,
 '240-479': 3125,
 '480-719': 390,
 '720-959': 97,
 '960-1199': 8,
 '1200-1439': 2,
 '1440-1679': 1,
 '1680-1919': 0,
 '1920-2159': 0,
 '2160-2399': 1}

In [22]:
lap0_sizes = samples_df[samples_df['lap'] == 0].groupby('sample').size().to_list()
c, b = np.histogram(lap0_sizes, bins=10)
b = np.round(b, 0)
r = {f'{int(b[i])}-{int(b[i+1])-1}':int(c[i]) for i in range(len(c))}
print('max:', max(lap0_sizes))
print('min:', min(lap0_sizes))
r

max: 446
min: 1


{'1-45': 674,
 '46-89': 103,
 '90-133': 73,
 '134-178': 46,
 '179-223': 20,
 '224-267': 7,
 '268-311': 4,
 '312-356': 1,
 '357-401': 1,
 '402-445': 3}

In [23]:
samples_dfp = samples_df[(samples_df['lap'] != 0) & (samples_df['lap'] != 8) ].copy()
samples_dfp.shape

(1627374, 8)

In [24]:
samples_dfp['lap'].unique()

array([1, 2, 3, 4, 5, 6, 7])

In [25]:
sizes = samples_dfp.groupby('sample').size()
max(sizes), min(sizes)

# Nro de samples por LAP
ids_mins = []
for l in range(1, 8):
    v = samples_dfp[samples_dfp['lap'] == l]['sample'].unique()
    sizes = samples_dfp[samples_dfp['lap'] == l].groupby('sample').size().to_dict()
    print('lap:' ,l, len(v), max(sizes), min(sizes))
    ids_mins.append(sizes)

lap: 1 911 4943 0
lap: 2 900 4960 5
lap: 3 847 4988 12
lap: 4 768 4965 14
lap: 5 528 5008 18
lap: 6 110 5012 172
lap: 7 17 4389 363


In [26]:
samples_dfp.groupby('sample').size().to_dict()

{0: 427,
 1: 433,
 2: 291,
 3: 451,
 4: 426,
 5: 447,
 6: 467,
 7: 454,
 8: 262,
 9: 397,
 10: 401,
 11: 430,
 12: 481,
 13: 470,
 14: 444,
 15: 434,
 16: 450,
 17: 465,
 18: 402,
 19: 468,
 20: 464,
 22: 443,
 23: 420,
 24: 125,
 25: 331,
 26: 431,
 27: 446,
 28: 449,
 29: 450,
 30: 458,
 31: 427,
 32: 447,
 33: 415,
 34: 426,
 35: 450,
 36: 435,
 37: 475,
 38: 449,
 39: 357,
 40: 439,
 41: 453,
 42: 402,
 43: 404,
 44: 415,
 45: 452,
 46: 427,
 47: 437,
 48: 480,
 49: 272,
 50: 451,
 51: 125,
 52: 263,
 53: 414,
 54: 420,
 55: 454,
 56: 480,
 57: 429,
 58: 899,
 59: 450,
 60: 427,
 61: 446,
 62: 433,
 63: 438,
 64: 501,
 65: 433,
 66: 467,
 67: 467,
 68: 429,
 69: 457,
 70: 464,
 71: 458,
 72: 443,
 73: 123,
 74: 406,
 75: 525,
 76: 436,
 77: 684,
 78: 454,
 79: 448,
 80: 459,
 81: 411,
 82: 438,
 83: 495,
 84: 272,
 86: 414,
 87: 465,
 88: 433,
 89: 457,
 90: 443,
 91: 412,
 92: 444,
 93: 370,
 94: 467,
 95: 456,
 96: 429,
 97: 270,
 98: 622,
 99: 418,
 100: 432,
 101: 388,
 102: 42

En base al analisis se determina que lap 0 no posee datos que ayuden en la prediccion por lo que seran eliminados

In [27]:
fff = samples_dfp.groupby('sample').size().to_list()
print('max', max(fff))
print('min', min(fff))
ccc, bbb = np.histogram(fff, bins=1, range=(0, 220))
bbb = np.round(bbb, 0)
{f'{int(bbb[i])}-{int(bbb[i+1])-1}':int(ccc[i]) for i in range(len(ccc))}

max 2328
min 1


{'0-219': 434}

In [28]:
d = samples_dfp.groupby('sample').size().to_dict()
ids_to_drop = {}
for k, v in d.items():
    if v <= 220:
        ids_to_drop[k]=v
        
len(ids_to_drop), ids_to_drop

(434,
 {24: 125,
  51: 125,
  73: 123,
  153: 14,
  160: 189,
  161: 145,
  182: 38,
  194: 165,
  198: 13,
  217: 220,
  261: 77,
  265: 3,
  283: 1,
  288: 171,
  304: 8,
  320: 42,
  337: 1,
  462: 149,
  500: 214,
  517: 101,
  526: 141,
  527: 125,
  530: 179,
  550: 18,
  552: 62,
  558: 139,
  593: 122,
  621: 171,
  626: 55,
  630: 22,
  632: 3,
  635: 29,
  653: 93,
  673: 210,
  676: 154,
  682: 77,
  691: 36,
  695: 35,
  705: 181,
  707: 1,
  710: 43,
  716: 93,
  723: 45,
  731: 146,
  734: 22,
  743: 1,
  745: 24,
  746: 15,
  759: 9,
  764: 6,
  776: 31,
  780: 128,
  788: 183,
  793: 9,
  800: 1,
  801: 125,
  802: 3,
  805: 14,
  810: 24,
  814: 8,
  820: 1,
  824: 1,
  840: 207,
  894: 109,
  929: 198,
  937: 132,
  944: 187,
  957: 198,
  980: 212,
  1031: 168,
  1051: 125,
  1060: 218,
  1077: 218,
  1078: 32,
  1080: 170,
  1085: 20,
  1098: 3,
  1106: 133,
  1137: 102,
  1138: 179,
  1143: 14,
  1145: 20,
  1155: 32,
  1184: 219,
  1195: 96,
  1211: 3,
  1233: 11,